In [ ]:
import os
import time
import json
import openai
from openai import OpenAI
from google.colab import userdata

# Initialize OpenAI client (assume API key is set in environment)
# client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
# client = userdata.get('OPENAI_API_KEY')
# openai.api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
# Define the book title and delivery date
TITLE_OF_THE_BOOK = "Agent Systems and Applications"
DATE_OF_DELIVERY = "2025-07-15"

In [ ]:
# Define the tools (functions) for the assistant
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_bookstores",
            "description": "Get the list of bookstores that have the specified book.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "The title of the book."},
                },
                "required": ["title"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "send_cfp",
            "description": "Send a Call For Proposal (CFP) to a bookstore for the book with a desired delivery date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "seller": {"type": "string", "description": "The name of the seller/bookstore."},
                    "title": {"type": "string", "description": "The title of the book."},
                    "delivery_date": {"type": "string", "description": "The desired delivery date."},
                },
                "required": ["seller", "title", "delivery_date"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "accept_proposal",
            "description": "Accept the best proposal from a seller and get confirmation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "seller": {"type": "string", "description": "The name of the seller/bookstore."},
                    "price": {"type": "number", "description": "The agreed price."},
                },
                "required": ["seller", "price"],
            },
        }
    },
]

In [ ]:
# Create the OpenAI Assistant (Client Agent)
assistant = client.beta.assistants.create(
    name="Client Agent",
    instructions=f"""You are a client agent purchasing the book '{TITLE_OF_THE_BOOK}' with desired delivery date '{DATE_OF_DELIVERY}'.
    Follow these steps:
    1. Use the 'get_bookstores' tool to get the list of sellers for the book title.
    2. If no sellers, inform that the book is not available.
    3. For each seller, use the 'send_cfp' tool to send a CFP with the title and delivery date.
    4. Collect all proposals (each proposal will return price and delivery_date).
    5. Select the proposal with the lowest price (ignore delivery_date for selection).
    6. If there is a best proposal, use 'accept_proposal' to accept it.
    7. Output the final result, including acceptance or failure.
    Simulate the behaviors and outputs similar to a JADE agent, printing status messages.""",
    model="gpt-4o",
    tools=tools,
)

In [ ]:
# Simulated function implementations (to mimic DF and sellers)
def get_bookstores(title):
    # Simulate DF response
    if title == TITLE_OF_THE_BOOK:
        return json.dumps({"sellers": ["Seller1", "Seller2", "Seller3"]})
    else:
        return json.dumps({"sellers": []})

def send_cfp(seller, title, delivery_date):
    # Simulate seller proposals (price and delivery date)
    proposals = {
        "Seller1": {"price": 20.0, "delivery_date": "2023-05-15"},
        "Seller2": {"price": 18.5, "delivery_date": "2023-05-16"},
        "Seller3": {"price": 22.0, "delivery_date": "2023-05-14"},
    }
    if seller in proposals and title == TITLE_OF_THE_BOOK:
        return json.dumps(proposals[seller])
    else:
        return json.dumps({"error": "No proposal available"})

def accept_proposal(seller, price):
    # Simulate acceptance and inform (return delivery confirmation)
    return json.dumps({"confirmation": f"Book will be delivered by {seller} on 2023-05-15 at price {price}."})

In [ ]:
# Map function names to implementations
available_functions = {
    "get_bookstores": get_bookstores,
    "send_cfp": send_cfp,
    "accept_proposal": accept_proposal,
}

# Create a thread for the conversation
thread = client.beta.threads.create()

# Start the process by adding a user message
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Start the book purchase process for 'Agent Systems and Applications'.",
)

# Create a run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

/tmp/ipython-input-1938676011.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/tmp/ipython-input-1938676011.py:12: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
/tmp/ipython-input-1938676011.py:19: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [ ]:
# Poll the run status and handle tool calls
while run.status in ["queued", "in_progress", "requires_action"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    time.sleep(1)  # Wait a bit before polling again

    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            function_to_call = available_functions.get(function_name)

            if function_to_call:
                function_response = function_to_call(**function_args)
                tool_outputs.append(
                    {
                        "tool_call_id": tool_call.id,
                        "output": function_response,
                    }
                )

        # Submit tool outputs back to the run
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs,
        )

/tmp/ipython-input-3303971826.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
/tmp/ipython-input-3303971826.py:25: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.submit_tool_outputs(


In [ ]:
# Once completed, retrieve and print the messages
messages = client.beta.threads.messages.list(thread_id=thread.id)
for message in messages.data:
    if message.role == "assistant":
        print(message.content[0].text.value)

The proposal from Seller2 has been accepted successfully. The book 'Agent Systems and Applications' will be delivered by Seller2 for a price of $18.5. The delivery is scheduled for 2023-05-15. 

The purchase process is now complete! If you have any further questions or need assistance, feel free to ask.
I've received the following proposals from the sellers:

1. Seller1: Price = $20.0, Delivery Date = 2023-05-15
2. Seller2: Price = $18.5, Delivery Date = 2023-05-16
3. Seller3: Price = $22.0, Delivery Date = 2023-05-14

The proposal from Seller2 offers the lowest price at $18.5. Let's proceed to accept this proposal.
We have found the following sellers offering the book 'Agent Systems and Applications':
1. Seller1
2. Seller2
3. Seller3

Next, I will send a Call For Proposal (CFP) to each of these sellers with the desired delivery date '2025-07-15'. Let's proceed with that.
Let's begin by finding the list of bookstores that are selling the book 'Agent Systems and Applications'. I'll star

/tmp/ipython-input-2466194763.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread.id)
